# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
print(len(city_data_df))

# Display sample data
city_data_df.head()

564


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fontanelas,38.8472,-9.4372,17.62,58,100,3.32,PT,1712797324
1,1,tiksi,71.6872,128.8694,-14.48,89,100,1.58,RU,1712797327
2,2,piton saint-leu,-21.2196,55.3151,22.06,73,0,3.09,RE,1712797331
3,3,yellowknife,62.4560,-114.3525,1.34,86,20,3.60,CA,1712797274
4,4,margaret river,-33.9500,115.0667,17.86,55,96,5.19,AU,1712797337


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))


# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

20


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,whitehorse,60.7161,-135.0538,8.43,31,75,7.72,CA,1712797364
43,43,mariehamn,60.0973,19.9348,5.92,65,0,4.63,AX,1712797468
92,92,iwanai,42.9744,140.5089,11.02,49,62,1.50,JP,1712797629
101,101,murun-kuren,49.6342,100.1625,5.52,42,82,2.09,MN,1712797659
178,178,vegreville,53.5001,-112.0518,9.51,25,1,3.87,CA,1712797817


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,whitehorse,CA,60.7161,-135.0538,31,
43,mariehamn,AX,60.0973,19.9348,65,
92,iwanai,JP,42.9744,140.5089,49,
101,murun-kuren,MN,49.6342,100.1625,42,
178,vegreville,CA,53.5001,-112.0518,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df

Starting hotel search
whitehorse - nearest hotel: Town & Mountain Hotel
mariehamn - nearest hotel: Hotell Esplanad
iwanai - nearest hotel: ホチルいのう
murun-kuren - nearest hotel: No hotel found
vegreville - nearest hotel: Vegreville Suites
fort mcmurray - nearest hotel: Nomad Hotel and Suites
constantia - nearest hotel: Hotel Chérica
rawson - nearest hotel: Hotel Deportivo
shohimardon - nearest hotel: No hotel found
brooks - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
fort st. john - nearest hotel: Fort St. John Motor Inn
nemuro - nearest hotel: お宿エクハシ
lloydminster - nearest hotel: Central Suite Hotel Knight's Inn
levuka - nearest hotel: No hotel found
altai - nearest hotel: Altai hotel 40000-100000T
summerside - nearest hotel: Comme Chez Vous
chibougamau - nearest hotel: No hotel found
anacapri - nearest hotel: Villa
dryden - nearest hotel: Holiday Inn Express


,City,Country,Lat,Lng,Humidity,Hotel Name
12,whitehorse,CA,60.7161,-135.0538,31,Town & Mountain Hotel
43,mariehamn,AX,60.0973,19.9348,65,Hotell Esplanad
92,iwanai,JP,42.9744,140.5089,49,ホチルいのう
101,murun-kuren,MN,49.6342,100.1625,42,No hotel found
178,vegreville,CA,53.5001,-112.0518,25,Vegreville Suites
211,fort mcmurray,CA,56.7268,-111.3810,46,Nomad Hotel and Suites
212,constantia,RO,44.1833,28.6500,60,Hotel Chérica
299,rawson,AR,-43.3002,-65.1023,62,Hotel Deportivo
322,shohimardon,UZ,39.9839,71.8028,40,No hotel found
330,brooks,CA,50.5834,-111.8851,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)